In [1]:
import sys

from morpheus.project.types.geometry.MultiPolygon import MultiPolygon

sys.path.insert(0, '..')

#!{sys.executable} -m pip install folium
#!{sys.executable} -m pip install openapi_core


In [2]:
from morpheus.project.types.discretization.spatial import Rotation, LengthUnit, Grid
from morpheus.project.types.geometry import Polygon

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96609040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(20)
length_unit = LengthUnit.meters()

grid = Grid.cartesian_from_polygon(
  polygon=polygon,
  n_rows=1000,
  n_cols=1000,
  rotation=rotation,
)
polygon

Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96609040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon')

## Performance of ActiveCells Calculation with a LineString 

In [3]:
from morpheus.project.types.geometry import LineString

linestring_within = LineString(
  type='LineString',
  coordinates=[
    (13.923, 50.965),
    (13.924, 50.965),
  ]
)

In [4]:
%%time
from morpheus.project.types.discretization.spatial import ActiveCells

ac = ActiveCells.from_linestring(linestring=linestring_within, grid=grid)
len(ac)

CPU times: user 273 ms, sys: 3.7 ms, total: 276 ms
Wall time: 226 ms


295

## Performance of ActiveCells Calculation with a Polygon

In [5]:
from morpheus.project.types.geometry import Polygon

polygon_within = Polygon(
  type='Polygon',
  coordinates=[[
    (13.923, 50.965),
    (13.924, 50.9655),
    (13.925, 50.965),
    (13.925, 50.966),

    (13.923, 50.966),
    (13.923, 50.965),
  ]]
)

In [6]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m


In [7]:
%%time
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)
len(active_cells)

CPU times: user 593 ms, sys: 9.92 ms, total: 603 ms
Wall time: 621 ms


318046

## Performance of ActiveCells Calculation with a MultiPolygon

In [8]:
multi_polygon_within = MultiPolygon(
  type='MultiPolygon',
  coordinates=[
    [[
      (13.923, 50.965),
      (13.924, 50.9655),
      (13.925, 50.965),
      (13.925, 50.966),
      (13.923, 50.966),
      (13.923, 50.965),
    ]],
    [[
      (13.9235, 50.9655),
      (13.924, 50.966),
      (13.9245, 50.9655),
      (13.9235, 50.9655),
    ]]
  ]
)

active_cells = ActiveCells.from_multipolygon(multipolygon=multi_polygon_within, grid=grid)

## Time to generate the outline of the active cells

In [9]:
active_cells = ActiveCells.from_polygon(polygon=polygon_within, grid=grid)

In [10]:
%%time
active_cells_outline = active_cells.outline_to_geojson(grid=grid)
print(active_cells_outline)

Feature(geometry=Polygon(coordinates=(((13.922996603410859, 50.96592678236522), (13.922996625639682, 50.965926788625694), (13.922998282676476, 50.96592716845478), (13.92299837734852, 50.96592719511799), (13.922998468943359, 50.965927230952445), (13.92299855657493, 50.96592727561149), (13.922998639395509, 50.965927328663106), (13.922998716603914, 50.96592738959409), (13.922998787453253, 50.96592745781501), (13.922998851258146, 50.965927532665916), (13.922998907401366, 50.96592761342272), (13.922998955339796, 50.96592769930421), (13.922998994609694, 50.96592778947958), (13.922999024831174, 50.9659278830765), (13.922999045711881, 50.96592797918956), (13.922999057049823, 50.965928076888964), (13.922999058735318, 50.9659281752296), (13.922999050752061, 50.96592827326016), (13.92299903317728, 50.965928370032316), (13.92299900618099, 50.96592846460992), (13.922998970024345, 50.96592855607805), (13.922998925057112, 50.96592864355188), (13.922998808951782, 50.96592884445043), (13.92299801521172

In [11]:
import folium

m = folium.Map(location=[50.9655, 13.9235], zoom_start=15)
folium.GeoJson(polygon.as_geojson()).add_to(m)
folium.GeoJson(polygon_within.as_geojson()).add_to(m)
folium.GeoJson(active_cells_outline.as_geojson()).add_to(m)
m.fit_bounds(m.get_bounds())
m